# German data

In [1]:
import os
import pandas as pd

In [2]:
with open("res_germany.json") as file:
    data = pd.read_json(file)
data.shape

(76101, 17)

In [3]:
data.head()

,asin,brand,categories,description,helpful,imUrl,overall,price,related,reviewText,reviewTime,reviewerID,reviewerName,salesRank,summary,title,unixReviewTime
0,B00005O66O,WMF,"[[Home & Kitchen, Kitchen & Dining, Kitchen Ut...",This narrow lifter is just one of the high-qua...,"[0, 0]",http://ecx.images-amazon.com/images/I/315H1HXT...,5,19.23,"{'bought_together': ['B00005O66M', 'B00005O66E...",All the WMF articles I have bought are very st...,"11 26, 2011",A1QH07ZORJLW1A,Marlene,{'Kitchen & Dining': 109323},Good kitchen tool,Profi Plus 11-Inch Stainless Steel Spatula,1322265600
1,B00005O66O,WMF,"[[Home & Kitchen, Kitchen & Dining, Kitchen Ut...",This narrow lifter is just one of the high-qua...,"[0, 0]",http://ecx.images-amazon.com/images/I/315H1HXT...,5,19.23,"{'bought_together': ['B00005O66M', 'B00005O66E...",My straight spatula was so old (and battered) ...,"03 13, 2010",A3QJQQZTKFV7BJ,"Sandy ""WR Gma""",{'Kitchen & Dining': 109323},Perfect kitchen accessory,Profi Plus 11-Inch Stainless Steel Spatula,1268438400
2,B00005O66O,WMF,"[[Home & Kitchen, Kitchen & Dining, Kitchen Ut...",This narrow lifter is just one of the high-qua...,"[4, 4]",http://ecx.images-amazon.com/images/I/315H1HXT...,5,19.23,"{'bought_together': ['B00005O66M', 'B00005O66E...",I purchased this spatula six years ago when I ...,"12 27, 2007",ADNPUV2B9CLEQ,C. Bullman,{'Kitchen & Dining': 109323},WMF: Quality products,Profi Plus 11-Inch Stainless Steel Spatula,1198713600
3,B00005O66O,WMF,"[[Home & Kitchen, Kitchen & Dining, Kitchen Ut...",This narrow lifter is just one of the high-qua...,"[0, 0]",http://ecx.images-amazon.com/images/I/315H1HXT...,5,19.23,"{'bought_together': ['B00005O66M', 'B00005O66E...","This spatula is a nice heavy, durable weight &...","10 20, 2013",AI02N0QYMMFR5,Stacy Coukoulis Julien,{'Kitchen & Dining': 109323},Nice SS spatula,Profi Plus 11-Inch Stainless Steel Spatula,1382227200
4,B0000CFPKC,Melitta,"[[Home & Kitchen, Kitchen & Dining, Dining & E...",62955 Features: -Set of 100 coffee filters.-Te...,"[0, 1]",http://ecx.images-amazon.com/images/I/41PbskA3...,5,3.31,NaN,I grabbed these on a whim at the store one day...,"07 5, 2010",A1Z54EM24Y40LL,csm,{'Home &amp; Kitchen': 889762},The best I've used!,Basket Coffee Filter in White,1278288000


## DATA CLEANING

In [4]:
# categories are like [[Health & Personal Care,...]], we are making them be "Health & Personal Care,..." 
def convert_cat():
    s = data["categories"]
    val = []
    for l in list(s): val.append("".join(l[0]))
    s_new = pd.Series(val, s.index.values)
    data["categories"]=s_new
convert_cat()

In the next few cells delete products with same name as swiss brands, but they represent actually other brand.

In [5]:
#this code is used to explore categories of brands
grouped_data = data.groupby("brand")
values = list(grouped_data.get_group("Junghans")["categories"])
prod = {}
for i,a in enumerate(values):
    if not a in list(prod.keys()):
        prod[a]=grouped_data.get_group("Junghans").iloc[i]["asin"]
prod

{'Clothing, Shoes & JewelryWomen': 'B000KB0QEE'}

### Join categories

In [6]:
data_new_categories = data.copy(deep=True)

In [8]:
fashion = ['Clothing, Shoes & JewelryNew Arrivals', 'Sports & OutdoorsAccessoriesSport Watches', 'Clothing', 
           'Shoes & JewelryLuxury Watches','Clothing, Shoes & JewelryLuggage & Travel GearLuggageSuitcases', 
           'Clothing, Shoes & JewelryLuggage & Travel GearLuggageGarment Bags',
      'Clothing, Shoes & JewelryJewelry: International Shipping Available', 
           'Clothing, Shoes & JewelryLuggage & Travel GearTravel AccessoriesPacking Organizers',
           'Clothing, Shoes & JewelryTTag Heuer', 'Clothing, Shoes & JewelryWomenWatchesWrist Watches', 
           'Clothing, Shoes & JewelryFashion Watches',
           'Clothing, Shoes & JewelryLuggage & Travel GearBackpacksCasual Daypacks', 
           'Clothing, Shoes & JewelryLuggage & Travel GearTravel AccessoriesTravel Wallets',
       'Clothing, Shoes & JewelryLuggage & Travel Gear', 'Clothing, Shoes & JewelryMenWatchesWrist Watches', 
           'Clothing, Shoes & JewelryWWenger','Clothing, Shoes & JewelryLuggage & Travel GearBriefcases', 
           'Clothing, Shoes & JewelryMenAccessoriesWallets, Card Cases & Money OrganizersCard & ID Cases',
       'Amazon Fashion', 'Clothing, Shoes & JewelryWomenWatchesWatch Bands', 
           'Arts, Crafts & SewingBeading & Jewelry-MakingJewelry-Making Tools & AccessoriesJewelry-Making Tools',
          'Clothing, Shoes & JewelryMenAccessoriesWallets, Card Cases & Money OrganizersMoney Clips', 
           'Office ProductsOffice & School SuppliesCarrying Cases',
           'Clothing, Shoes & JewelryLuggage & Travel GearLuggageTravel Duffels', 
           'Clothing, Shoes & JewelryLuggage & Travel GearLuggageTravel Totes',
           'Clothing, Shoes & JewelryLuggage & Travel GearLuggageCarry-Ons',   
           'Clothing, Shoes & JewelryWomenAccessoriesWallets, Card Cases & Money OrganizersWallets', 
           'Clothing, Shoes & JewelryGirls', 'Clothing, Shoes & JewelryWomen',
           'Clothing, Shoes & JewelryNovelty, Costumes & MoreBand & Music FanAccessories',
           'Video GamesPCAccessoriesHeadsets', 'Clothing, Shoes & JewelryShoes & Accessories: International Shipping Available',
           'Clothing, Shoes & JewelryOmega Watches', 'Clothing, Shoes & JewelryCollectible Watches',
           'Clothing, Shoes & JewelryLuggage & Travel GearFashion Waist Packs', 
           'Clothing, Shoes & JewelryLuggage & Travel GearTravel AccessoriesLuggage Locks',
           'Clothing, Shoes & JewelryBabyBaby GirlsClothingLayette Sets',
           'Office ProductsOffice & School SuppliesCarrying CasesPad Holders','Clothing, Shoes & JewelrySwatch Watches',
           'Clothing, Shoes & JewelryMenWatchesPocket Watches', 
           'Clothing, Shoes & JewelryLuggage & Travel GearTravel Accessories',
           'Office ProductsOffice & School SuppliesBinders & Binding SystemsBindersPadfolio Ring Binders',
           'Clothing, Shoes & JewelryLuggage & Travel GearMessenger Bags',
           'Clothing, Shoes & JewelryLuggage & Travel GearUmbrellas', 'Clothing, Shoes & JewelryMen',
           'Clothing, Shoes & JewelryLuggage & Travel GearTravel AccessoriesLuggage Straps',
           'Clothing, Shoes & JewelryGifts', 'Clothing, Shoes & JewelryLuxury Watches',
           'Office ProductsOffice & School SuppliesCalendars, Planners & Personal OrganizersPersonal Organizers',
            'Clothing, Shoes & JewelryMenWatchesWatch Bands'
          ] 
kitchen_tools = ['Home & KitchenKitchen & DiningKitchen Knives & Cutlery AccessoriesCarving Knives & ForksCarving Sets',
                 'Home & KitchenKitchen & DiningKitchen Knives & Cutlery AccessoriesBoning & Fillet Knives',
                 'Tools & Home ImprovementPower & Hand ToolsHand ToolsKnivesUtility Knives', 
                 'Sports & OutdoorsOutdoor GearCamping & HikingKnives & ToolsFolding Knives',
         "Home & KitchenKitchen & DiningKitchen Knives & Cutlery AccessoriesChef's Knives", 
                 'Home & KitchenKitchen & DiningKitchen Utensils & GadgetsCooking UtensilsSpatulas',
                 'Tools & Home ImprovementPower & Hand ToolsHand ToolsMultitools & AccessoriesMultitools',
                 'Home & KitchenKitchen & DiningKitchen Knives & Cutlery AccessoriesCarving Knives & ForksCarving Knives',
         'Home & KitchenKitchen & DiningKitchen Knives & Cutlery AccessoriesBread & Serrated KnivesSerrated Utility Knives', 
        'Home & KitchenKitchen & DiningKitchen Knives & Cutlery AccessoriesAsian KnivesSantoku Knives',
                 'Home & KitchenKitchen & DiningKitchen Knives & Cutlery AccessoriesBread & Serrated KnivesBread Knives',
        'Sports & OutdoorsHunting & FishingHuntingHunting & Tactical KnivesHunting KnivesFixed Blade Hunting Knives', 
                 'Home & KitchenKitchen & DiningKitchen Utensils & GadgetsCan Openers',
        'Home & KitchenKitchen & DiningKitchen Knives & Cutlery Accessories', 
                 'Home & KitchenKitchen & DiningKitchen Knives & Cutlery AccessoriesParing Knives',
        'Home & KitchenKitchen & Dining', 
'Tools & Home ImprovementSafety & SecurityPersonal Protective EquipmentHand & Arm ProtectionLab', 
                 'Safety & Work GlovesCut Resistant Gloves', 
                'Home & KitchenKitchen & DiningKitchen Knives & Cutlery AccessoriesCutlery Sets',
                 'Home & KitchenKitchen & DiningKitchen Knives & Cutlery AccessoriesKnife Sharpeners',
                'Home & KitchenKitchen & DiningKitchen Utensils & GadgetsSpecialty Tools & GadgetsSeafood Tools', 
                 'Home & KitchenKitchen & DiningKitchen Knives & Cutlery AccessoriesCutlery SetsBoxed Knife Sets',
                'Home & KitchenKitchen & DiningKitchen Knives & Cutlery AccessoriesKnife Blocks & StorageKnife Blocks',
                 'Home & KitchenKitchen & DiningKitchen Knives & Cutlery AccessoriesUtility Knives',
               'Tools & Home ImprovementPower & Hand ToolsHand ToolsKnivesKnife Sets',
                 'Sports & OutdoorsHunting & FishingFishingFishing Tools & Maintenance EquipmentKnives',
                'Tools & Home ImprovementPower & Hand ToolsHand ToolsKnivesPocket Knives & Folding Knives', 
                 'Tools & Home ImprovementPower & Hand ToolsHand ToolsKnivesPocket Knives & Folding Knives',
                'Home & KitchenKitchen & DiningKitchen Knives & Cutlery AccessoriesKnife Blocks & StorageKnife Cases', 
                 'Holders & Protectors',
                'Tools & Home ImprovementPower & Hand ToolsHand ToolsKnives', 
                 'Home & KitchenKitchen & DiningKitchen Knives & Cutlery AccessoriesSteak Knives',
                 'Home & KitchenKitchen & DiningKitchen Knives & Cutlery AccessoriesCutlery SetsKnife Block Sets',
                 'Home & KitchenKitchen & DiningKitchen Utensils & GadgetsGraters, Peelers & SlicersMandolines & Slicers',
                 'Sports & OutdoorsOutdoor GearCamping & HikingKnives & ToolsFixed-Blade Knives',
                 'Home & KitchenKitchen & DiningKitchen Utensils & GadgetsKitchen AccessoriesToothpick Holders',
                 'Tools & Home ImprovementHardwareCabinet HardwareLatches', 'Home & KitchenKitchen & DiningSmall Appliances',
                 'Home & KitchenKitchen & DiningKitchen Utensils & GadgetsCheese ToolsSpreaders', 
                 'Home & KitchenKitchen & DiningStorage & Organization', 
                 'Home & KitchenStorage & OrganizationClothing & Closet StorageHangers', 
                 'Home & KitchenKitchen & DiningStorage & OrganizationLunch Boxes & BagsLunch Boxes',
                 'Home & KitchenKitchen & DiningSmall Appliance Parts & Accessories', 
                 'Home & KitchenKitchen & DiningKitchen Knives & Cutlery AccessoriesCutlery SetsSteak Sets',
                 'Home & KitchenKitchen & DiningDining & EntertainingFlatwareKnives', 
                 'Office ProductsOffice & School SuppliesStaplers & PunchesManual StaplersDesktop Staplers',
                 'Home & KitchenKitchen & DiningWater Coolers & FiltersReplacement Water Filters',
                 'Tools & Home ImprovementHardwareDoor Hardware & LocksHandlesets', 
        'Home & KitchenKitchen & DiningKitchen Knives & Cutlery AccessoriesKnife Blocks & StorageMagnetic Knife Strips',
                 'Home & KitchenKitchen & DiningKitchen Utensils & GadgetsGraters, Peelers & SlicersPeelers',
                 'Office ProductsOffice & School SuppliesForms, Recordkeeping & Money HandlingKey Tags & Chains',
                 'Home & KitchenKitchen & DiningKitchen Knives & Cutlery AccessoriesSpecialty KnivesGarnishing Knives',
                 'Sports & OutdoorsHunting & FishingHuntingHunting & Tactical KnivesHunting KnivesFolding Hunting Knives',
                 'Home & KitchenKitchen & DiningKitchen Knives & Cutlery AccessoriesCutlery Accessories',
                 'Home & KitchenKitchen & DiningSmall Appliance Parts & AccessoriesFood Processor Parts & Accessories',
                 'Sports & OutdoorsHunting & FishingHuntingHunting & Tactical KnivesHunting Knives',
                 'Home & KitchenKitchen & DiningDining & EntertainingCups, Mugs & SaucersCoffee Cups & Mugs',
                 'Tools & Home ImprovementHardwareNails, Screws & FastenersNails', 
                 'Tools & Home ImprovementPower & Hand ToolsHand ToolsWrenchesHex Keys',
                 'Office ProductsOffice & School SuppliesEducation & CraftsStudents Scissors',
                 'Tools & Home ImprovementPower & Hand ToolsHand Tools', 
                 'Home & KitchenKitchen & DiningKitchen Utensils & GadgetsFruit & Vegetable ToolsMelon Ballers',
                 'Home & KitchenKitchen & DiningKitchen Knives & Cutlery AccessoriesSpecialty KnivesTomato Knives',
                 'Home & KitchenKitchen & DiningKitchen Utensils & GadgetsPasta & Pizza ToolsPizza Cutters',
                 'Home & KitchenKitchen & DiningKitchen Utensils & GadgetsGraters, Peelers & Slicers',
                 'Tools & Home ImprovementRough PlumbingToilet PartsToilet Flush Valves',
        'Home & KitchenKitchen & DiningDining & EntertainingFlatwareServing UtensilsCake, Pie & Pastry Servers',
                 'Arts, Crafts & SewingCraft SuppliesWoodcrafts', 
                 'Tools & Home ImprovementPower & Hand ToolsHand ToolsMagnetic Sweepers',
                 'Tools & Home ImprovementPower & Hand ToolsHand ToolsMagnetic Sweepers',
                 'Office ProductsOffice & School SuppliesWriting & Correction SuppliesPens & RefillsPen Refills',
                 'Kitchen & Dining', 'Home & KitchenKitchen & DiningKitchen Knives & Cutlery AccessoriesSpecialty Knives',
                 'Tools & Home ImprovementKitchen & Bath FixturesBathroom FixturesToilets & Toilet PartsToilet Tanks',
                 'Home & KitchenKitchen & DiningKitchen Utensils & GadgetsMeasuring Tools & ScalesSpoons',
                 'Home & KitchenKitchen & DiningKitchen Knives & Cutlery AccessoriesShears',
                 'Tools & Home ImprovementBuilding SuppliesBuilding MaterialsRaw Building MaterialsMortar Mix',
                 'Home & KitchenKitchen & DiningKitchen Knives & Cutlery AccessoriesCutting Boards',
                 'Tools & Home ImprovementHardwareDoor Hardware & LocksLock Replacement Parts',
                 'Tools & Home ImprovementPower & Hand ToolsTool OrganizersTool Pouches',
                 'Tools & Home ImprovementSafety & SecurityPersonal Protective EquipmentHand & Arm ProtectionLab, Safety & Work GlovesSafety Work Gloves',
                 'Tools & Home ImprovementKitchen & Bath FixturesBathroom FixturesBathroom Sink & Bathtub AccessoriesSink & Bathtub DrainsSink Drains',
                 'Home & KitchenKitchen & DiningKitchen Utensils & GadgetsCooking UtensilsLadles',
                 'Home & KitchenBeddingBed PillowsTravel Pillows', 
                 'Industrial & ScientificLab & Scientific ProductsLab Supplies & ConsumablesLab UtensilsLab Knives & Scalpels',
                 'Home & KitchenKitchen & DiningKitchen Utensils & GadgetsFruit & Vegetable ToolsZesters & Reamers',
                 'Sports & OutdoorsHunting & FishingFishingFishing Tools & Maintenance EquipmentPliers & Multi-Tools',
                 'Tools & Home ImprovementSafety & SecurityPersonal Protective EquipmentHand & Arm ProtectionLab, Safety & Work GlovesCut Resistant Gloves',
                 'Tools & Home ImprovementKitchen & Bath FixturesBathroom FixturesBathroom Hardware & InstallationBathroom Fixture Hardware Parts'
               
                ]
espresso_machines = ['Home & KitchenKitchen & DiningCoffee, Tea & EspressoEspresso Machines', 
'Home & KitchenKitchen & DiningSmall Appliance Parts & AccessoriesCoffee & Espresso Machine Parts & AccessoriesCoffee Machine AccessoriesCoffee Filters',
                    'Home & KitchenKitchen & DiningCoffee, Tea & EspressoEspresso MachinesSemi-Automatic Espresso Machines',
                     'Home & KitchenKitchen & DiningCoffee, Tea & EspressoCoffee Makers',
                     'Home & KitchenKitchen & DiningSmall AppliancesCoffee, Tea & Espresso AppliancesMilk Frothers',
                     'Home & KitchenKitchen & DiningCoffee, Tea & EspressoEspresso MachinesSuper-Automatic Espresso Machines',
                     'Home & KitchenKitchen & DiningSmall Appliance Parts & AccessoriesCoffee & Espresso Machine Parts & AccessoriesCoffee & Espresso Machine Cleaning Products',
                     'Home & KitchenKitchen & DiningCoffee, Tea & EspressoEspresso Machine & Coffeemaker Combos',
                     'Home & KitchenKitchen & DiningSmall Appliance Parts & AccessoriesCoffee & Espresso Machine Parts & AccessoriesEspresso Machine AccessoriesMilk Frothing Pitchers',
                     'Home & KitchenKitchen & DiningCoffee, Tea & Espresso', 
                     'Home & KitchenKitchen & DiningSmall Appliance Parts & AccessoriesCoffee & Espresso Machine Parts & AccessoriesEspresso Machine Accessories',
                     'Home & KitchenKitchen & DiningCoffee, Tea & EspressoCoffee MakersSingle-Serve Brewers',
                     'Home & KitchenKitchen & DiningSmall Appliance Parts & AccessoriesCoffee & Espresso Machine Parts & AccessoriesEspresso Machine AccessoriesTampers',
                     'Home & KitchenKitchen & DiningCoffee, Tea & EspressoCoffee MakersCoffee Machines',
                     'Home & KitchenKitchen & DiningKitchen Knives & Cutlery AccessoriesKnife Blocks & StorageKnife Cases, Holders & Protectors'
                    ]
pets=["Pet SuppliesDogsTreatsCookies, Biscuits & SnacksSnacks", 
      'Pet SuppliesDogsHealth SuppliesFlea, Lice & Tick ControlFlea Shampoos',
      'Pet SuppliesDogsHealth SuppliesFlea, Lice & Tick ControlFlea Drops',
      'Pet SuppliesCatsHealth SuppliesFlea, Lice & Tick ControlFlea Drops'
     ]
food = ['Grocery & Gourmet Food', 'Health & Personal CareSports Nutrition', 'Baby', 
        'Grocery & Gourmet FoodCandy & ChocolateBars', 'Office & School Supplies',
        'Grocery & Gourmet FoodCandy & ChocolateGummy Candy','Grocery & Gourmet FoodBeveragesTeaChai',
        'Health & Personal CareSports NutritionEndurance & EnergyCandy & Mints',
        'Grocery & Gourmet FoodBeveragesCoffeeSingle-Serve Capsules',
        'Grocery & Gourmet FoodCandy & ChocolateChocolate Truffles',
        'Grocery & Gourmet FoodBeveragesWaterSparkling & Mineral Water',
        'Grocery & Gourmet FoodBeveragesCoffeeCoffee Substitutes','Grocery & Gourmet FoodBaby FoodsCereal',
        'Grocery & Gourmet FoodBeveragesEnergy Drinks', 
        'Grocery & Gourmet FoodBeveragesPowdered Drink MixesInstant Breakfast Drinks',
        'Grocery & Gourmet FoodCandy & ChocolateHard Candy',
        'Grocery & Gourmet FoodBeveragesTeaHerbal',
        'Baby ProductsFeedingBottle-FeedingBottles','Grocery & Gourmet FoodBeveragesCoffeeInstant Coffee',
        'Grocery & Gourmet FoodBeveragesWater', 'Grocery & Gourmet FoodBeveragesHot Cocoa',
        'Grocery & Gourmet FoodBeveragesTeaTea Samplers'
       ]
electronic_and_games = ['ElectronicsComputers & AccessoriesCables & AccessoriesComputer Speakers',
                    'ElectronicsComputers & AccessoriesCables & AccessoriesKeyboards',
             'ElectronicsComputers & AccessoriesCables & AccessoriesKeyboard & Mouse Combos',
                    'ElectronicsComputers & AccessoriesLaptop & Netbook Computer AccessoriesDocking Stations',
             'ElectronicsComputers & AccessoriesCables & AccessoriesMice', 
                    'ElectronicsComputers & AccessoriesTouch Screen Tablet AccessoriesCases & SleevesCases',
             'Health & Personal CareHousehold SuppliesHousehold BatteriesCoin & Button Cell', 
                    'Video GamesPCAccessoriesGaming Mice',
            'Cell Phones & AccessoriesCasesBasic Cases', 'ElectronicsComputers & AccessoriesWebcams', 
                    'BeautySkin CareFaceCreams & MoisturizersFluids & LotionsLotions',
             'ElectronicsComputers & AccessoriesCables & AccessoriesHeadsets & MicrophonesPC Headsets', 
                    'Cell Phones & AccessoriesAccessoriesHeadsetsBluetooth Headsets',
             'ElectronicsComputers & AccessoriesLaptop & Netbook Computer AccessoriesBags & CasesBackpacks',
                    'ElectronicsComputers & AccessoriesCables & AccessoriesHeadsets & Microphones',
             'ElectronicsComputers & AccessoriesLaptop & Netbook Computer AccessoriesBags & CasesSleeves & Slipcases',
                    'ElectronicsComputers & AccessoriesCables & AccessoriesInput DevicesTouch Pads',
             'ElectronicsAccessories & SuppliesAudio & Video AccessoriesHeadphones', 
                    'ElectronicsComputers & AccessoriesLaptop & Netbook Computer AccessoriesBags & CasesMessenger Bags',
             'ElectronicsComputers & AccessoriesTouch Screen Tablet AccessoriesSkins & Decals', 
                    'Video GamesPCAccessories',
                    'ElectronicsPortable Audio & VideoMP3 Players & AccessoriesMP3 Player AccessoriesSpeaker Systems',
              'Toys & GamesGamesGame Collections', 'ElectronicsHome AudioRadiosInternet Radios', 
        'Office ProductsOffice & School SuppliesDesk Accessories & Workspace OrganizersMouse Pads & Wrist RestsMouse Pads',
              'Computers', 'ElectronicsGPS & NavigationGPS System AccessoriesCases', 'Toys & GamesBuilding ToysBuilding Sets', 
                    'Toys & GamesBuilding Toys',
               'ElectronicsComputers & AccessoriesCables & AccessoriesCables & InterconnectsUSB Cables',    
                    'Video GamesPCAccessoriesGaming Keyboards',
              'ElectronicsHome AudioStereo ComponentsSpeakersSpeaker Systems',       
            'ElectronicsCamera & PhotoVideo SurveillanceSurveillance CamerasSimulated Cameras',
            'Video GamesMore SystemsGameCubeHardwareAccessories',
            'ElectronicsAccessories & SuppliesAudio & Video AccessoriesCables & InterconnectsPower Cables',
                    'ElectronicsHome AudioStereo ComponentsSpeakers', 
                    'ElectronicsAccessories & SuppliesAudio & Video AccessoriesRemote ControlsTV Remote Controls',
             'ElectronicsComputers & AccessoriesCables & Accessories', 'Video GamesPCAccessoriesControllersJoysticks',
                    'ElectronicsComputers & AccessoriesCables & AccessoriesInput DevicesTrackballs',
                    'ElectronicsAccessories & SuppliesAudio & Video AccessoriesConnectors & Adapters',
                'Cell Phones & AccessoriesAccessoriesBluetooth Speakers', 
                    'ElectronicsComputers & AccessoriesCables & AccessoriesCleaning & RepairRepair Kits',
                'Toys & GamesLearning & EducationScience', 'Video GamesPCAccessoriesHeadsets',
                    'ElectronicsPortable Audio & VideoMP3 Players & AccessoriesMP3 Player AccessoriesCases',
                'ElectronicsComputers & AccessoriesComputer ComponentsMemory', 
                    'Home & KitchenBathBathroom AccessoriesBathroom Accessory Sets', 'ElectronicsComputers & Accessories',
                'ElectronicsComputers & AccessoriesLaptop & Netbook Computer AccessoriesChargers & Adapters',
                    'ElectronicsComputers & AccessoriesComputer Components',
                    'ElectronicsAccessories & SuppliesAudio & Video AccessoriesRemote ControlsRemote-Control Extenders',
                   'Toys & GamesToy Remote Control & Play VehiclesVehicle Playsets',  
            'ElectronicsTelevision & VideoStreaming Media Players', 'Toys & GamesToy Remote Control & Play VehiclesPlay Vehicles',
                    'ElectronicsPortable Audio & VideoMP3 Players & AccessoriesMP3 Player AccessoriesRemote Controls',
                    'ElectronicsAccessories & SuppliesBatteries, Chargers & AccessoriesAC Adapters',
              'Video GamesPCAccessoriesControllersGamepads',  'ElectronicsHome AudioStereo ComponentsTurntables',
                    'Toys & GamesNovelty & Gag ToysLight-Up Toys', 'All Electronics', 
            'ElectronicsAccessories & SuppliesAudio & Video Accessories', 
                    'ElectronicsAccessories & SuppliesBatteries, Chargers & AccessoriesPower Converters',
                    'ElectronicsComputers & AccessoriesRouters',
                    'ElectronicsComputers & AccessoriesTouch Screen Tablet AccessoriesCases & Sleeves',
                    'ElectronicsComputers & AccessoriesCables & AccessoriesVideo Projector AccessoriesLamps',
                 'ElectronicsComputers & AccessoriesLaptop & Netbook Computer AccessoriesBatteries',
                    'Office ProductsOffice ElectronicsTelephones & AccessoriesAudio Conferencing',
                    'ElectronicsComputers & AccessoriesCables & AccessoriesHeadsets & MicrophonesPC Microphones',
                    'ElectronicsComputers & AccessoriesCables & AccessoriesInput DevicesNumeric Keypads',
                    'ElectronicsComputers & AccessoriesTouch Screen Tablet Accessories',
                    'ElectronicsComputers & AccessoriesTouch Screen Tablet AccessoriesStands',
                    'ElectronicsCamera & PhotoVideo SurveillanceSurveillance CamerasHidden Cameras',
                    'Cell Phones & AccessoriesAccessoriesHeadsetsWired Headsets',
                'ElectronicsCamera & PhotoBags & CasesCamera Cases',  
                    'Office ProductsOffice ElectronicsPresentation ProductsPresentation PointersPresentation Remotes',
                    'Toys & Games', 'Video GamesMore SystemsPlayStation 2HardwareAccessories',
                    'Cell Phones & AccessoriesAccessoriesChargersTravel Chargers', 
                    'ElectronicsAccessories & SuppliesAudio & Video AccessoriesHeadphone Accessories',
                    'ElectronicsComputers & AccessoriesPDAs, Handhelds & AccessoriesPDA & Handheld AccessoriesKeyboards & Styluses',
                    'ElectronicsComputers & AccessoriesLaptop & Netbook Computer AccessoriesCooling Pads',
                    'ElectronicsAccessories & SuppliesTelephone AccessoriesBatteries',
                    'ElectronicsAccessories & SuppliesBatteries, Chargers & AccessoriesPDUs', 
                    'Toys & GamesSports & Outdoor PlaySportsBasketball',
                    'ElectronicsComputers & AccessoriesPDAs, Handhelds & AccessoriesPDA & Handheld AccessoriesCases',
                    'Toys & GamesAction Figures & StatuesAction Figures',
                    'ElectronicsComputers & AccessoriesLaptop & Netbook Computer AccessoriesBags & Cases',
                        'Toys & GamesBaby & Toddler ToysActivity Play Centers'
             ]
sports = ['Sports & OutdoorsExercise & FitnessStrength Training EquipmentDumbbells', 
          'Sports & OutdoorsExercise & FitnessAccessories',
         'Sports & OutdoorsAccessories', 'Sports & OutdoorsCyclingBike Tools & MaintenanceGrease', 
          'Sports & OutdoorsAccessoriesSports Water Bottles'
         'Sports & OutdoorsAccessoriesSports Sunglasses', 'Sports & OutdoorsOutdoor GearCamping & Hiking', 
         'Sports & OutdoorsCyclingHydrationWater Bottles', 
          'Sports & OutdoorsOutdoor GearCamping & HikingBackpacks & BagsHiking Daypacks', 
          'Sports & OutdoorsOutdoor GearCamping & HikingPersonal CareInsect Repellent',
          'Sports & OutdoorsTeam SportsCurlingBrooms & Brushes', 'Sports & OutdoorsBoating & Water SportsSwimmingGoggles',
          'Sports & OutdoorsExercise & FitnessAccessoriesExercise Balls & Accessories',
          'Sports & OutdoorsCyclingBike Tools & MaintenanceCleaners', 
          'Sports & OutdoorsOutdoor GearCamping & HikingCamp BeddingBlankets',
          'Sports & OutdoorsCyclingBike Tools & MaintenanceBike Lube', 
          'Sports & OutdoorsAccessoriesSports Water Bottle Accessories',
          'Sports & OutdoorsExercise & FitnessStrength Training EquipmentBack Machines','Sports & Outdoors',
          'Sports & OutdoorsExercise & FitnessAccessoriesGloves', 'Sports & OutdoorsAction SportsSkateboardingHelmets',
          'Sports & OutdoorsHunting & FishingTactical & DutyTactical Bags & Packs',
          'Sports & OutdoorsFan ShopSports SouvenirsTrading Cards',
          'Sports & OutdoorsOutdoor GearCamping & HikingHydration',
          'Sports & OutdoorsAccessoriesSports Water Bottles',
          'Sports & OutdoorsAccessoriesSports Sunglasses'
         ]
personal_care= ['Health & Personal CarePersonal CareDeodorants & AntiperspirantsDeodorants', 
                'BeautySkin CareFaceCreams & Moisturizers',
          'Health & Personal CareHealth CareMassage & RelaxationAromatherapyOils', 'BeautyBath & BodyCleansersBody Washes',
          'BeautySkin CareFaceCreams & MoisturizersFluids & LotionsLotions', "BeautyFragranceWomen'sEau de Parfum", 
                'BeautySkin CareBodyMoisturizersLotions', 
           "Health & Personal CareHealth CareSmoking CessationNicotine Gums & Lozenges", 
                'Health & Personal CareHealth CareCough & ColdCoughing & Sore ThroatsCough Drops',
            'Health & Personal CareHousehold Supplies', 'Health & Personal CareMedical Supplies & Equipment',
                'Health & Personal CareHealth CareMassage & RelaxationMassage Oils',
            'Health & Personal CarePersonal CareEye CareEye Drops, Lubricants & Washes', 
                'BeautySkin CareHands & NailsHand Creams & Lotions',
        'Health & Personal Care', 'BeautyTools & AccessoriesBags & CasesCosmetic Bags', 
                'Health & Personal CareMedical Supplies & EquipmentDaily Living Aids',
               'BeautyMakeupMakeup RemoverEyes', 'BeautySkin CareFaceCreams & MoisturizersNight Creams', 
                'BeautyBath & BodyCleansersSoaps',
               'Baby ProductsBathing & Skin CareSkin CareDiaper Creams', 'BeautySkin CareBodyMoisturizersOils', 
               'Health & Personal CareHealth CareDigestion & NauseaAntacids', 
                'BeautySkin CareFaceCreams & MoisturizersDay Creams', 
                'Industrial & ScientificProfessional Dental SuppliesOral Hygiene & Preventive CareToothpastes',
                'Arts, Crafts & SewingCraft SuppliesPaper & Paper CraftsPapermaking', 
                'BeautyHair CareStyling ProductsHair Styling Serums', 'BeautySkin CareBodyMoisturizersCreams', 
                'BeautySkin CareFaceTreatments & Masks','Baby ProductsBathing & Skin CareSkin CareLotions',
                'Health & Personal CareVitamins & Dietary SupplementsSupplements', 'BeautySkin CareMaternity',
                'Health & Personal CarePersonal CareShaving & Hair RemovalShaving Creams, Lotions & SoapsCreams', 
                'Health & Personal CareVitamins & Dietary SupplementsHerbal Supplements', 
                'Health & Personal CareMedical Supplies & EquipmentHome Tests', 'BeautySkin CareEyesDark Circle Treatments',
                'BeautyTools & AccessoriesBags & CasesToiletry Bags',
                'Health & Personal CarePersonal CareLip CareBalms & Moisturizers', 'BeautyHair CareHair & Scalp Treatments',
                'BeautyBath & BodyScrubs & Body TreatmentsBody Mud', 
                'Health & Personal CareHealth CareFirst Aid','BeautyTools & AccessoriesNail ToolsClippers & Trimmers',
                'Health & Personal CarePersonal CareEye CareContact Lens CareRewetting Drops','BeautyBath & BodyBathOils',
                'Health & Personal CarePersonal Care', "BeautyFragranceWomen'sEau de Toilette",
                'Health & Personal CarePersonal CareEye CareContact Lens CareSoaking Solutions',
                'Health & Personal CareMedical Supplies & EquipmentDiabetes Care', 'BeautyHair CareShampoos',
                'BeautySkin CareEyesCreams', 
                'Health & Personal CareMedical Supplies & EquipmentDaily Living AidsEating & Drinking Aids',
                'Health & Personal CareSports NutritionProteinPowdersWhey',
                'Health & Personal CareHealth CareCough & ColdCold & Flu Relief','BeautyMakeupLipsLip Glosses',
                'BeautySkin CareFaceTreatments & MasksMasks',
                'Health & Personal CarePersonal CareOral HygieneBreath FreshenersBreath Mints','Baby ProductsBathing & Skin Care',
                'Health & Personal CareVitamins & Dietary SupplementsSupplementsAntioxidantsLutein',
                'Health & Personal CareVitamins & Dietary SupplementsSupplementsDigestive SupplementsProbiotics','All Beauty',
                'Health & Personal CareHealth CarePain RelieversRubs & Ointments', 
                'Health & Personal CareHealth CareDigestion & NauseaLaxatives', 'Health & Personal CarePersonal CareEye Care',
                'BeautySkin CareFaceToners & Astringents', 'Health & Personal CareHealth CareFirst AidBurn Care',
                'BeautyHair CareConditioners', 
                'Health & Personal CareVitamins & Dietary SupplementsSupplementsDigestive SupplementsDietary Fibers',
                'Health & Personal CareHealth CareTherapeutic Skin Care',
                'Industrial & ScientificProfessional Medical SuppliesFluid Administration & CollectionBlood Lancets',
                'Health & Personal CarePersonal CareShaving & Hair RemovalHair RemovalTweezers',
                'Health & Personal CarePersonal CareEye CareContact Lens CareCleaners',
                'BeautyBath & BodyScrubs & Body TreatmentsBody Scrubs', 'BeautySkin CareFaceOils & Serums',
                'Health & Personal CarePersonal CareShaving & Hair RemovalAfter Shaves & Grooming AidsLotions & Tonics',
                'Health & Personal CarePersonal CareShaving & Hair RemovalAfter Shaves & Grooming Aids',
                'Health & Personal CarePersonal CareEye CareContact Lens Care',
                'Health & Personal CarePersonal CareFoot CareFoot Creams & Lotions', 
                'Health & Personal CareHealth CareDigestion & NauseaGas Relief', "BeautyFragranceMen'sEau de Toilette",
                "Industrial & Scientific", 'Health & Personal CareBaby & Child CarePersonal CareBaby Bath',
                'Industrial & ScientificProfessional Medical SuppliesFluid Administration & Collection',
                'Baby ProductsBathing & Skin CareGrooming & Healthcare Kits', 
                'Baby ProductsBathing & Skin CareSoaps & CleansersBody Wash','Baby ProductsBathing & Skin CareSkin CareOils',
                'Health & Personal CareMedical Supplies & EquipmentDiabetes CareLancets', 
                'Health & Personal CareMedical Supplies & EquipmentDiabetes CareTest StripsBlood Test Strips',
                'BeautySkin CareFaceExfoliators & ScrubsScrubs', 'Baby ProductsBathing & Skin CareSoaps & CleansersBar Soaps'
               ]
motors = ['AutomotiveOils & FluidsOilsMotor Oils', 'AutomotiveMotorcycle & PowersportsParts', 
          'AutomotiveMotorcycle & PowersportsPartsDrive TrainChains & SprocketsChains', 
          'AutomotiveMotorcycle & PowersportsPartsFiltersAir Filters',
          'Industrial & ScientificTapes, Adhesives & SealantsIndustrial SealantsThread Sealants',
          'AutomotiveCar CareExterior CareCleaners', 'AutomotiveMotorcycle & PowersportsProtective GearHelmets',
          'AutomotiveInterior AccessoriesKey Chains', 'AutomotiveOils & FluidsOilsHydraulic Oils',
          'AutomotiveReplacement PartsStarters & AlternatorsAlternators & GeneratorsVoltage Regulators',
          'Industrial & ScientificTapes, Adhesives & SealantsSilicone Adhesives',
          'Industrial & ScientificTapes, Adhesives & SealantsIndustrial Sealants',
          'AutomotiveOils & FluidsBody Repair & Restoration ChemicalsAdhesives'
         ]
gardening = ['Patio, Lawn & GardenGardening & Lawn CareSoils, Fertilizers & MulchesFertilizers & Plant Food', 
             'Patio, Lawn & GardenPest ControlLures',
             'Office ProductsOffice & School SuppliesPaperCopy & Multipurpose Paper',
             'Patio, Lawn & GardenGardening & Lawn CareWeed & Moss ControlWeed Killers'
             ]

In [9]:
for index, row in data_new_categories.iterrows():
    if row["categories"] in fashion: data_new_categories.set_value(index, 'categories', "fashion")
    if row["categories"] in kitchen_tools: data_new_categories.set_value(index, 'categories', "kitchen_tools")
    if row["categories"] in espresso_machines: data_new_categories.set_value(index, 'categories', "espresso_machines")
    if row["categories"] in pets: data_new_categories.set_value(index, 'categories', "pets")
    if row["categories"] in food: data_new_categories.set_value(index, 'categories', "food")
    if row["categories"] in electronic_and_games: data_new_categories.set_value(index, 'categories', "electronic_and_games")
    if row["categories"] in sports: data_new_categories.set_value(index, 'categories', "sports")
    if row["categories"] in personal_care: data_new_categories.set_value(index, 'categories', "personal_care")
    if row["categories"] in motors: data_new_categories.set_value(index, 'categories', "motors")
    if row["categories"] in gardening: data_new_categories.set_value(index, 'categories', "gardening")
data_new_categories = data_new_categories[
                                         (data_new_categories["categories"]=="fashion") |
                                         (data_new_categories["categories"]=="kitchen_tools") |
                                         (data_new_categories["categories"]=="espresso machines") |
                                         (data_new_categories["categories"]=="pets") |
                                         (data_new_categories["categories"]=="food") |
                                         (data_new_categories["categories"]=="electronic_and_games") |
                                         (data_new_categories["categories"]=="sports") |
                                         (data_new_categories["categories"]=="personal_care") |
                                         (data_new_categories["categories"]=="motors") |
                                         (data_new_categories["categories"]=="gardening")
                                         ]

In [10]:
data_new_categories["categories"].drop_duplicates()

0               kitchen_tools
12                       food
13              personal_care
355      electronic_and_games
853                   fashion
4389                   sports
5321                   motors
18233               gardening
18660                    pets
Name: categories, dtype: object

## CATEGORY STATS - numeric ratings

In [16]:
#Mean rating of each brand
#make weighted statistic of data - take overall rate according to helpfulness (how helpful is rating and comment)
def calculate_review(data_frame):
    s=[]
    overall=[]
    overall_count=0
    for index, row in data_frame.iterrows():
        if row["helpful"][1]!=0: #and row["helpful"][1]>=3:
            overall.append(row["overall"]*(row["helpful"][0]/row["helpful"][1]))
            overall_count+=row["helpful"][0]/row["helpful"][1]
        #else: 
            #overall.append(row["overall"])
            #overall_count+=1
    if overall_count!=0: return sum(overall)/overall_count
    else: return 0

In [19]:
#mean ratings for each category
def mean_ratings(data):
    category_group = data.groupby("categories")
    s=[]
    for c in list(category_group.groups):
        group_cat = pd.DataFrame(category_group.get_group(c)) 
        s.append(str(c)+", number of votes: "+str(len(group_cat))+ ", mean: "+ str(calculate_review(group_cat)))
    for i in range (5): print (s[i])
mean_ratings(data)

, number of votes: 1, mean: 5.0
Health & Personal CareHealth CareDigestion & NauseaAntacids, number of votes: 1, mean: 5.0
Home & KitchenKitchen & DiningCookwareSteamers, number of votes: 9, mean: 4.760000000000001
Home & KitchenKitchen & DiningSmall AppliancesOvens & ToastersToasters, number of votes: 213, mean: 3.8847583851122796
Home & KitchenKitchen & DiningKitchen Utensils & GadgetsKitchen AccessoriesTrivets, number of votes: 17, mean: 4.783505154639176


In [14]:
#what are the most and the least popular products within each category
import operator
def popular_products(data):
    categories = data["categories"].drop_duplicates()
    grouped_cat = data.groupby("categories")
    for cat in categories:
        category = pd.DataFrame(grouped_cat.get_group(cat))
        products = category["asin"].drop_duplicates()
        grouped_products = category.groupby("asin")
        reviews={}
        for p in products:
            product = pd.DataFrame(grouped_products.get_group(p))
            if calculate_review(product)!=0: reviews[list(product["title"])[0]]=calculate_review(product)
        sorted_reviews = sorted(reviews.items(), key=operator.itemgetter(1), reverse=True)
        print ("CATEGORY: ",cat, ", BEST REVIEWS: ", sorted_reviews[0:3], "\nCATEGORY: ",cat, ", WORST REVIEWS: ", sorted_reviews[-4:-1], "\n")
popular_products(data)

CATEGORY:  Home & KitchenKitchen & DiningKitchen Utensils & GadgetsCooking UtensilsSpatulas , BEST REVIEWS:  [('J.A. Henckels Twin Pure Silicone Spatula', 5.0), ('Profi Plus 11-Inch Stainless Steel Spatula', 5.0), ('J.A. Henckels International Silicone Turner', 5.0)] 
CATEGORY:  Home & KitchenKitchen & DiningKitchen Utensils & GadgetsCooking UtensilsSpatulas , WORST REVIEWS:  [('WMF Profi Plus  13-3/4 Inch Silicone Wok Turner', 3.6666666666666665), ('WMF Profi Plus Non-Stick Wok Turner, 13-3/4 Inch', 3.0), ('WMF Profi Plus 13 Inch Silicone Long Turner', 2.3333333333333335)] 

CATEGORY:  Home & KitchenKitchen & DiningDining & EntertainingCups, Mugs & SaucersCoffee Cups & Mugs , BEST REVIEWS:  [('Basket Coffee Filter in White', 5.0), ('Krups F472 Duo Replacement Water Filter, Set of 4', 3.5), ('Flavor Pores Coffee Filter (Set of 100)', 2.5447941888619856)] 
CATEGORY:  Home & KitchenKitchen & DiningDining & EntertainingCups, Mugs & SaucersCoffee Cups & Mugs , WORST REVIEWS:  [('Basket Cof

In [15]:
mean_ratings(data_new_categories)
print("\n\n")
popular_products(data_new_categories)
print("\n\n")
#popular_brands(data_new_categories)

personal_care, number of votes: 1416, mean: 4.482921053327992
fashion, number of votes: 13, mean: 5.0
gardening, number of votes: 459, mean: 3.917705220773438
sports, number of votes: 261, mean: 4.496832304116583
kitchen_tools, number of votes: 5153, mean: 4.046612336958192



CATEGORY:  kitchen_tools , BEST REVIEWS:  [('Zwilling J.A. Henckels Four Star 5-1/2-Inch High Carbon Stainless Steel Boning Knife', 5.0), ('Profi Plus 11-Inch Stainless Steel Spatula', 5.0), ('J.A. Henckels International Forged Synergy 5-Inch Hollow-Edge Santoku', 5.0)] 
CATEGORY:  kitchen_tools , WORST REVIEWS:  [('J.A. Henckels International Henckels Serrated Poultry Shears', 1.2014388489208634), ('WMF Profi Plus 12-1/2-Inch Butterfly Turner', 1.0), ('J.A. HENCKELS INTERNATIONAL 12-Piece Fine Edge Pro Block Knife Set', 1.0)] 

CATEGORY:  food , BEST REVIEWS:  [('Knorr Feischmecker Curry Sauce Mix- 1 pc', 5.0), ('Melitta Cone Coffee Filters, White, No. 2, 100-Count Filters (Pack of 6)', 5.0), ('Melitta 75412 Buz

In [ ]:
data_new_categories["categories"].drop_duplicates()